In [1]:
import os

In [2]:
%pwd

'c:\\Madhu\\2025_NLPDataScientist\\UKJobs\\Hiscox\\insuranceClaim\\End-to-end-MLOps-AWS-CICD-Pipeline\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Madhu\\2025_NLPDataScientist\\UKJobs\\Hiscox\\insuranceClaim\\End-to-end-MLOps-AWS-CICD-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from mlopsProject.constants import *
from mlopsProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config


In [18]:
import os
from mlopsProject import logger
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek, SMOTEENN
import pandas as pd

In [35]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        data.drop(columns=["product_var_3", "marital_status", "occupation", "location","family_history_3","employment_type"], inplace=True, axis=1)
        X = data.drop('claim_status', axis=1)
        y = data['claim_status']
        smt = SMOTEENN(random_state=42,sampling_strategy='minority')
        X_res, y_res = smt.fit_resample(X, y)
        X_train, X_test, y_train, y_test = train_test_split(X_res,y_res,test_size=0.2,random_state=42)

        X_train.to_csv(os.path.join(self.config.root_dir, "X_train.csv"),index = False)
        X_test.to_csv(os.path.join(self.config.root_dir, "X_test.csv"),index = False)

        y_train.to_csv(os.path.join(self.config.root_dir, "y_train.csv"),index = False)
        y_test.to_csv(os.path.join(self.config.root_dir, "y_test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(X_train.shape)
        logger.info(X_test.shape)
        logger.info(y_train.shape)
        logger.info(y_test.shape)

        print(X_train.shape)
        print(X_test.shape)
        print(y_train.shape)
        print(y_test.shape)  

In [36]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-07-17 04:02:58,341: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-17 04:02:58,343: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-17 04:02:58,350: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-17 04:02:58,352: INFO: common: created directory at: artifacts]
[2024-07-17 04:02:58,354: INFO: common: created directory at: artifacts/data_transformation]
[2024-07-17 04:02:59,605: INFO: 2025303918: Splited data into training and test sets]
[2024-07-17 04:02:59,606: INFO: 2025303918: (10550, 34)]
[2024-07-17 04:02:59,607: INFO: 2025303918: (2638, 34)]
[2024-07-17 04:02:59,608: INFO: 2025303918: (10550,)]
[2024-07-17 04:02:59,609: INFO: 2025303918: (2638,)]
(10550, 34)
(2638, 34)
(10550,)
(2638,)
